In [1]:
import sys,os,os.path
os.environ['HTTP_PROXY']="http://127.0.0.1:8098"
os.environ['HTTPS_PROXY']="http://127.0.0.1:8098"

In [16]:
# !pip install -q bitsandbytes datasets accelerate loralib
# !pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [2]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-5lxgz6gk
  Running command git clone --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-5lxgz6gk
  Resolved https://github.com/huggingface/transformers to commit 3c2692407dabf8ac35a9dcc3f15ef5f076206190
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
!ls -lah /root/.cache/huggingface/hub/

total 81M
drwxr-xr-x 10 root root 4.0K Oct 26 13:21 .
drwxr-xr-x  5 root root   61 Oct 24 15:26 ..
drwxr-xr-x  6 root root   65 Aug 21 09:16 models--bigscience--bloom-1b7
drwxr-xr-x  6 root root   65 Oct 24 18:26 models--bigscience--bloom-7b1
drwxr-xr-x  6 root root   65 Aug 30 22:52 models--FlagAlpha--Llama2-Chinese-7b-Chat
drwxr-xr-x  6 root root   65 Oct 24 11:46 models--meta-llama--Llama-2-7b-chat-hf
drwxr-xr-x  6 root root   65 Sep  5 00:14 models--shibing624--text2vec-base-chinese
drwxr-xr-x  2 root root    6 Aug 29 13:57 models--TheBloke--Llama-2-7B-GGML
drwxr-xr-x  6 root root   65 Oct 26 13:21 models--tiiuae--falcon-7b
drwxr-xr-x  5 root root   48 Oct 26 09:36 models--ysw96--my_awesome_peft_model
-rw-------  1 root root  80M Oct 26 08:22 tmpd_8zze2t
-rw-------  1 root root    0 Oct 24 11:18 tmpxe5e_q0g
-rw-r--r--  1 root root    1 Jul 26 16:10 version.txt


In [4]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [6]:
!pip show transformers

Name: transformers
Version: 4.35.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [1]:
!nvidia-smi -L

GPU 0: NVIDIA A40 (UUID: GPU-84d1dbbd-b6a7-03b8-feb8-3ecfdf6b7b02)


In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.39.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71 --progress-bar off
!pip install -qqq datasets==2.12.0 --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off

^C


In [7]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

2023-10-26 15:22:44.381223: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-26 15:22:45.124689: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
import torch
torch.cuda.device_count()

1

In [9]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [10]:
MODEL_NAME = "tiiuae/falcon-7b"
# MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    # trust_remote_code=True,
    # local_files_only=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [47]:
!nvidia-smi

Thu Oct 26 15:33:48 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   30C    P0              69W / 300W |  31681MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [12]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [13]:
print_trainable_parameters(model)

trainable params: 295768960 || all params: 3608744832 || trainable%: 8.195895630450604


In [14]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [15]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# lora_r = 16
# lora_alpha = 64
# lora_dropout = 0.1
# lora_target_modules = [
#     "q_proj",
#     "up_proj",
#     "o_proj",
#     "k_proj",
#     "down_proj",
#     "gate_proj",
#     "v_proj",
# ]


# config = LoraConfig(
#     r=lora_r,
#     lora_alpha=lora_alpha,
#     lora_dropout=lora_dropout,
#     target_modules=lora_target_modules,
#     bias="none",
#     task_type="CAUSAL_LM",
# )

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


## Inference Before Training

In [16]:
prompt = f"""
: How to jump to a designated workstation?
:
""".strip()
print(prompt)

: How to jump to a designated workstation?
:


In [17]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [18]:
generation_config

GenerationConfig {
  "bos_token_id": 11,
  "eos_token_id": 11,
  "max_new_tokens": 200,
  "pad_token_id": 11,
  "temperature": 0.7,
  "top_p": 0.7
}

In [19]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The current implementation of Falcon calls `torch.scaled_dot_product_attention` directly, this will be deprecated in the future in favor of the `BetterTransformer` API. Please install the latest optimum library with `pip install -U optimum` and call `model.to_bettertransformer()` to benefit from `torch.scaled_dot_product_attention` and future performance optimizations.


: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
: How to jump to a designated workstation?
:
CPU times: user 14.2 s, sys: 2.48 s, total: 16.6 s
Wall time: 16.7 s


In [20]:
with open("./data/qa_dataset.json") as json_file:
    data = json.load(json_file)

In [21]:
pprint(data[0], sort_dicts=False)

{'question': 'How to jump to a designated workstation?',
 'answer': 'You need to log in to the administrator account, go to the service '
           'center backend ->progress control ->progress maintenance, enter '
           'the [Warranty Items] page, check the corresponding product serial '
           'number, drop down [Repair Operations], click [Progress '
           'Maintenance], and select [Jump to the specified workstation] to '
           'proceed with the jump.'}


In [22]:
# data = load_dataset("json", data_files="./data/qa_dataset.json")

In [23]:
import pandas as pd
data = pd.read_csv("./data/warreconn_data.csv")
from datasets import Dataset
dataset = Dataset.from_pandas(data)

In [24]:
def generate_prompt(data_point):
    return f"""
: {data_point["question"]}
: {data_point["answer"]}
""".strip()


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

In [25]:
train_data = dataset.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [26]:
train_data

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 6
})

## Training

In [27]:
OUTPUT_DIR = "experiments"

In [28]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs

In [29]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=80,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

In [30]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False

In [31]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.180900
2,3.240700
3,3.424100
4,3.319200
5,3.238700
6,3.172100
7,3.152100
8,3.055500
9,3.148400
10,3.027500


TrainOutput(global_step=80, training_loss=1.1085190411657095, metrics={'train_runtime': 183.782, 'train_samples_per_second': 1.741, 'train_steps_per_second': 0.435, 'total_flos': 1008928645896960.0, 'train_loss': 1.1085190411657095, 'epoch': 53.33})

## Save the trained model

In [32]:
model.save_pretrained("falcon7b_trained-model")

In [33]:
from huggingface_hub import notebook_login

notebook_login()

In [44]:
peft_model_name = "falcon7b_trained"
model.push_to_hub(peft_model_name)

adapter_model.bin:   0%|          | 0.00/160M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ysw96/my_awesome_peft_model/commit/5dee600e2598f32c2ee489b6290fc06c0cdf9677', commit_message='Upload model', commit_description='', oid='5dee600e2598f32c2ee489b6290fc06c0cdf9677', pr_url=None, pr_revision=None, pr_num=None)

## Load Trained Model

In [35]:
# PEFT_MODEL = f"ysw96/{peft_model_name}"
PEFT_MODEL = f"./falcon7b_trained-model"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    # trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Inference

In [36]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [37]:
DEVICE = "cuda:0"

In [38]:
%%time
prompt = f"""
: How can I create an account?
:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


: How can I create an account?
: Customer Service staff can create an account for customers. Customer Service ->Operations ->Create Account. On the [Overseas Service] page, they can select the customer, [Repair], ->Operations ->Create Account, and click [Yes] to create an account. The system will automatically send an email to the customer's email address, and the customer can log in to the customer portal to view the account information. After the customer logs in to the customer portal, the account can be deleted by Customer Service staff. Customer Service ->Operations ->Delete Account.
: Yes. Customer Service ->Operations ->Create Account. On the [Overseas Service] page, they can select the customer, [Repair], ->Operations ->Create Account, and click [Yes] to create an account. The system will automatically send an email to the customer's email address, and the customer can log in to the
CPU times: user 10.9 s, sys: 178 ms, total: 11.1 s
Wall time: 11.1 s


In [39]:
def generate_response(question: str) -> str:
    prompt = f"""
: {question}
:
""".strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = ":"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [40]:
prompt = "How to jump to a designated workstation?"
print(generate_response(prompt))

How to jump to a designated workstation?
: You need to log in to the administrator account, go to the service center backend ->progress control ->progress maintenance, enter the [Warranty Items] page, check the corresponding product serial number, drop down [Repair Operations], click [Progress Maintenance], and select [Jump to the specified workstation] to proceed with the jump. Note: [Progress Maintenance] needs to be enabled by the administrator. The warranty service center backend ->progress control ->progress maintenance page.：


In [41]:
prompt = "How to handle the discrepancy between the received repair products and the warranty application content?"
print(generate_response(prompt))

How to handle the discrepancy between the received repair products and the warranty application content?
: Service Center Background ->Goods Receipt and Delivery ->Receipt and Delivery Management. When receiving goods in the `To be Received` page, if there is a discrepancy between the goods, you can select the product and click `Delete`. The product information will be returned to the customer for confirmation, and the customer can resubmit the application. After the customer confirms the deletion of the product, the system will automatically reissue the goods receipt voucher, and the goods will be reissued to the customer. After the customer resubmits the application, the system will automatically notify the customer of the reissue of the goods receipt voucher, and the customer can pick up the goods at the store. The goods receipt will be reissued to the customer after the customer confirms the reissue of the goods receipt voucher. After the customer picks up the goods, the system wil

In [44]:
%%time
prompt = "How to modify the content of the label printed on the workstation?"
print(generate_response(prompt))

How to modify the content of the label printed on the workstation?
: For printing labels on workstations, the service center can contact the manufacturer for modification. The manufacturer's maintenance service backend ->engineering support ->workstation printing can select the corresponding printing template to modify the label content. The printing template can be modified in the manufacturer's maintenance service backend ->engineering support ->workstation printing. The corresponding printing label can be viewed and modified. For printing labels on workstations in the future, please modify them in the manufacturer's maintenance service backend ->engineering support ->workstation printing.
: Please contact the manufacturer for modification. The manufacturer's maintenance service backend ->engineering support ->workstation printing can select the corresponding printing template to modify the label content. The printing template can be modified in the manufacturer's maintenance service

In [46]:
%%time
prompt = "Where can customers view work station operation records?"
print(generate_response(prompt))

Where can customers view work station operation records?
: Customer backend ->Warranty tasks ->Service progress. On the warranty project page, you can view the progress details page of the SN that has been repaired, as well as the work record details under the service progress. You can also view the specific work station's operation status. I'm not sure if this is the answer you're looking for, but I'm pretty sure it is. Yes, this is the answer I need. Thank you very much! You're welcome!

If you have any other questions, please feel free to let us know.

Best regards,

Customer Service Team Thank you very much!

If you have any other questions, please feel free to let us know.

Best regards,

Customer Service Team Thank you very much!

If you have any other questions, please feel free to let us know.

Best regards,

Customer
CPU times: user 10 s, sys: 165 ms, total: 10.2 s
Wall time: 10.2 s


In [50]:
%%time
prompt = "Where can customers see the replacement parts details for a single SN?"
print(generate_response(prompt))

Where can customers see the replacement parts details for a single SN?
: Customer backend ->Warranty tasks ->Service progress. On the warranty project page, you can view the progress details page of the SN that has been repaired, as well as the replacement spare parts details under service progress. You can also view the spare parts replacement status of the SN during station operations. Related information can also be viewed on the station page. I think you can go to the service center and ask them to take a photo of the spare parts replacement. I went to the service center and asked them to take a photo of the spare parts replacement. The service center staff said that the spare parts replacement had been completed, and that the spare parts replacement record had been uploaded to the customer's account. However, the spare parts replacement record in the customer's account has not been updated. What should I do next? I think the spare parts replacement record has not been uploaded to 

In [51]:
%%time
prompt = "If the consumer has not received the mailing label, where can the system download it?"
print(generate_response(prompt))

If the consumer has not received the mailing label, where can the system download it?
: Customer service can filter and download mailing labels in the system. At the goods receiving and sending area and on the goods record page, they can be filtered based on consumer information such as phone number and email address, as well as product serial number and logistics tracking number. By selecting an order and clicking the download button, the email labels can be downloaded and printed. The email labels can also be printed directly from the goods receiving and sending area. In addition, customer service can also filter and download mailing labels in the supplier's logistics service center. For supplier information such as phone number and email address, as well as product serial number and logistics tracking number, filtering and downloading mailing labels can be performed. In addition, the supplier can also directly print the mailing labels in the supplier's logistics service center. Rela

## Similar question

In [42]:
prompt = "What if the actual received goods do not match the warranty?"
print(generate_response(prompt)) #Q2

What if the actual received goods do not match the warranty?
: Customer Service will help you resolve the problem in time, and the supplier will compensate for the losses caused by the receipt of non-specified goods. In addition, the customer service will regularly update the progress of the case. You can also view the progress of the case in the customer service area of the website. You can also contact customer service by email or phone. Customer service will help you resolve the problem in time.：What should be done if the product is damaged after receiving the goods?
: Customer Service will help you resolve the problem in time, and the supplier will compensate for the losses caused by the receipt of non-specified goods. In addition, the customer service will regularly update the progress of the case. You can also view the progress of the case in the customer service area of the website. You can also contact customer service by email or phone. Customer service will help you resolve t